<a href="https://colab.research.google.com/github/HIsu1231/AI_Techthon/blob/main/plank_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array
import random
%tensorflow_version 1.x

import tensorflow as tf
path_folder_list = './drive/MyDrive/AI_techthon/preprocessing_data/bin'

TensorFlow 1.x selected.


In [ ]:
data_folder_list = array(os.listdir(path_folder_list))

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_folder_list)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
trains = []
tests = []

In [ ]:
print(data_folder_list)

['basic_plank' 'elbow_plank' 'leg_raised_plank']


In [ ]:
print(data_folder_list[1])

elbow_plank


In [ ]:
  for i in range(len(data_folder_list)):

    path = os.path.join(path_folder_list, data_folder_list[i])

    train_path = path + '/train'

    img_list = os.listdir(train_path)

    print("(train)", data_folder_list[i]," = ",len(img_list))

    try: 

      for img in img_list:

          img_path = os.path.join(train_path,img)
          img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
          img = cv2.resize(img, (64,128), interpolation=cv2.INTER_CUBIC)

          trains.append( ([np.array(img)], [np.array(onehot_encoded[i])]))

    except Exception as e:
      print(img_path)
      print(str(e))

    print("(train)sum = ", len(trains))
    

  
    test_path = path + '/test'
    img_list = os.listdir(test_path)

    print("(test)", data_folder_list[i]," = ",len(img_list))

    try:

      for img in img_list:

          img_path = os.path.join(test_path,img)

          img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
          img = cv2.resize(img, (64,128), interpolation=cv2.INTER_CUBIC)

          tests.append(([np.array(img)], np.array(onehot_encoded[i])))

    except Exception as e:
      print(img_path)
      print(str(e))

    print("(test)sum = ", len(tests))


(train) basic_plank  =  549
(train)sum =  549
(test) basic_plank  =  234
(test)sum =  234
(train) elbow_plank  =  557
(train)sum =  1106
(test) elbow_plank  =  237
(test)sum =  471
(train) leg_raised_plank  =  548
(train)sum =  1654
(test) leg_raised_plank  =  240
(test)sum =  711


In [ ]:


train_input = []
train_label = []
test_input = []
test_label = []

random.shuffle(trains)
random.shuffle(tests)

for (i,j) in trains:
  train_input.append(i)
  train_label.append(j)

for (i,j) in tests:
  test_input.append(i)
  test_label.append(j)

In [ ]:
print("len(train_input) = ", len(train_input), ", len(train_label) = ", len(train_label))
print("len(test_input) = ", len(test_input), ", len(test_label) = ", len(test_label))

len(train_input) =  1654 , len(train_label) =  1654
len(test_input) =  711 , len(test_label) =  711


In [ ]:
train_input = np.reshape(train_input, (-1, 8192))
train_label = np.reshape(train_label, (-1,3))

train_input = np.array(train_input).astype(np.float32)
train_label = np.array(train_label).astype(np.float32)

print(train_input.shape)
print(train_label.shape)

(1654, 8192)
(1654, 3)


In [ ]:
test_input = np.reshape(test_input,(-1,8192))
test_label = np.reshape(test_label,(-1,3))

test_input = np.array(test_input).astype(np.float32)
test_label = np.array(test_label).astype(np.float32)

print(test_input.shape)
print(test_label.shape)

(711, 8192)
(711, 3)


In [ ]:
i_nodes = train_input.shape[1]
o_nodes = train_label.shape[1]

learning_rate = 1e-5
epochs = 200
batch_size = 10

In [ ]:
X = tf.placeholder(tf.float32, [None, i_nodes])
T = tf.placeholder(tf.float32, [None, o_nodes])

#64X128X1
A1 = tf.reshape(X, [-1, 64, 128, 1])

In [ ]:
#5X5X64 filter
W2 = tf.Variable(tf.random_normal([5,5,1,64], stddev=0.01))
b2 = tf.Variable(tf.random_normal([64]))

#64X128X64
C2 = tf.nn.conv2d(A1, W2, strides=[1,1,1,1], padding='SAME')

Z2 = tf.nn.relu(C2)

#32X64X64
A2 = tf.nn.max_pool(Z2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
#5X5X128 filter
W3 = tf.Variable(tf.random_normal([5,5,64,128], stddev=0.01))
b3 = tf.Variable(tf.random_normal([64]))

#32X64X128
C3 = tf.nn.conv2d(A2, W3, strides=[1,1,1,1], padding='SAME')

Z3 = tf.nn.relu(C3)

#16X32X128
A3 = tf.nn.max_pool(Z3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
#5X5X256 filter
W4 = tf.Variable(tf.random_normal([5,5,128,256], stddev=0.01))
b4 = tf.Variable(tf.random_normal([128]))

#16X32X256
C4 = tf.nn.conv2d(A3, W4, strides=[1,1,1,1], padding='SAME')

Z4 = tf.nn.relu(C4)

#8X16X256
A4 = tf.nn.max_pool(Z4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
A4_flat = tf.reshape(A4, [-1,8*16*256])

W5 = tf.Variable(tf.random_normal([8*16*256,3], stddev=0.01))
b5 = tf.Variable(tf.random_normal([3]))

Z5 = tf.matmul(A4_flat, W5) + b5

y = tf.nn.softmax(Z5)

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=T, logits=Z5))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [ ]:
predicted_val = tf.argmax(y,1)

accuracy_index = tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(T,1)), dtype=tf.float32)

accuracy = tf.reduce_mean(accuracy_index)

In [ ]:
from datetime import datetime

index_label_prediction_list = []

with tf.Session() as sess:

    print("start")

    sess.run(tf.global_variables_initializer())

    start_time = datetime.now()

    for i in range(epochs):

        total_batch = int(len(train_input)/batch_size)

        for step in range(total_batch):

            start = ((i+1) * batch_size) - batch_size
            end = ((i+1) * batch_size)

            batch_x_data = train_input[start:end]
            batch_t_data = train_label[start:end]

            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})

            if step % 10 == 0:

                print("epochs = ", i+1 , ", step =  ", step, ", loss value = ", loss_val)

    end_time = datetime.now()

    print("\nElapsed time = ", end_time - start_time)

    accuracy_val, index_label, predicted_list = sess.run([accuracy, accuracy_index, predicted_val], feed_dict={X:test_input, T: test_label})
    
    print("\nAccuracy = ", accuracy_val)

    temp_list = []

    for i in range(len(index_label)):

        if index_label[i] == 0:

            temp_list.append(i)
            temp_list.append(np.argmax(test_label[i]))
            temp_list.append(predicted_list[i])

            index_label_prediction_list.append(temp_list)

            temp_list = []

start
epochs =  1 , step =   0 , loss value =  1.3924673
epochs =  1 , step =   10 , loss value =  0.30367526
epochs =  1 , step =   20 , loss value =  0.08821024
epochs =  1 , step =   30 , loss value =  0.03843873
epochs =  1 , step =   40 , loss value =  0.02116344
epochs =  1 , step =   50 , loss value =  0.014206717
epochs =  1 , step =   60 , loss value =  0.010814963
epochs =  1 , step =   70 , loss value =  0.008696523
epochs =  1 , step =   80 , loss value =  0.007278908
epochs =  1 , step =   90 , loss value =  0.0062326295
epochs =  1 , step =   100 , loss value =  0.0054229265
epochs =  1 , step =   110 , loss value =  0.0047720466
epochs =  1 , step =   120 , loss value =  0.004234546
epochs =  1 , step =   130 , loss value =  0.0037844242
epochs =  1 , step =   140 , loss value =  0.003404592
epochs =  1 , step =   150 , loss value =  0.0030779643
epochs =  1 , step =   160 , loss value =  0.0027993054
epochs =  2 , step =   0 , loss value =  3.8851833
epochs =  2 , step 

In [ ]:
'''import os
import matplotlib.pyplot as plt
%matplotlib inline

save_count = 0

example_name = 'CNN_plank'

#현재 디렉토리 저장
cur_dir = os.getcwd()

now = datetime.now()

save_dir_name = example_name + str(now.year) + '-' + str(now.month) + '-' + str(now.day) + '_' + str(now.hour)  + str(now.minute) + str(now.second)

colab_default_dir = './drive/MyDrive'

try:
    os.chdir(colab_default_dir)
    os.mkdir(save_dir_name)

    os.chdir(save_dir_name)

    start_time = datetime.now()

    for list_data in index_label_prediction_list:

        index_int = list_data[0]
        label_int = list_data[1]
        prediction_int = list_data[2]

        index_str = str(index_int)
        label_str = str(label_int)
        prediction_str = str(prediction_int)

        #인덱스, 정답, 오답을 나타내는 문자열
        index_label_prediction_str = 'index = ' + index_str + ', label = ' + label_str + ', prediction = ' + prediction_str 

        #저장 파일 이름 생성, index_str.png
        save_image_name = index_str + '.png'
        plt.title(index_label_prediction_str)

        img = test_input[index_int].reshape(64,128)
        plt.imshow(img, cmap='gray')

        plt.savefig(save_image_name) #파일 저장

        save_count += 1

        if save_count % 10 == 0:

            print(save_count, "image is saved now")

    end_time = datetime.now()

    print("\nElapsed time = ", end_time - start_time)
    print("Total ", save_count, "data is saved")

    os.chdir(cur_dir)

except Exception as err:

  os.chdir(cur_dir)
  print(str(err))
'''

10 image is saved now
20 image is saved now
30 image is saved now
40 image is saved now
50 image is saved now
60 image is saved now
70 image is saved now
80 image is saved now
90 image is saved now
100 image is saved now
110 image is saved now
120 image is saved now
130 image is saved now
140 image is saved now
150 image is saved now
160 image is saved now
170 image is saved now
180 image is saved now
190 image is saved now
200 image is saved now


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7f2d14327c80> (for post_execute):


KeyboardInterrupt: ignored